In [7]:
## lets deconstruct how we will work with audio. (transcription)
# whisper can directly be used to give audio files but in real world scenarios the audio would need pre processing
## process_audio.py has utilities for this. mainly ffmpeg is the core library for audio proc, use that and 
# other techniques to process audio into clean form. later at the output end also specific gpts can be used to summarise returned text 
# like a pipeline extract different flavors with using each gpt.

In [14]:
from constants import *
from utils import get_openai_client

In [9]:
from pathlib import Path

mp3_file = 'earning_call_morepen.mp3'
wav_file = 'EarningsCall.wav'
# ogg_file = 'processed_audio.ogg' 
ogg_file = 'processed_earning_call_morepen.ogg' # converted the file to ogg by using the utils 


In [10]:
from openai import OpenAI
client = get_openai_client()
client.timeout=50

def transcribe_audio(client:OpenAI,audio_file_path):
    with open(audio_file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create( model= "whisper-1", file=audio_file, response_format=)
    return transcription

In [11]:

# process to ogg converted the 1.2hr morepend recording to 7.3mb
# sample audio from openai was 50mb ->ogg 370kb (took 7s to transcribe)

audio_file = Path(Path(__name__).parent.resolve(),'data',ogg_file)

transcription = transcribe_audio(client=client,audio_file)

APITimeoutError: Request timed out.

In [ ]:
# sample audio openai 
transcription.__dict__.values()

NameError: name 'transcription' is not defined

In [ ]:
transcription.__dict__.values()

In [ ]:
! pip install pydub

In [ ]:
## lets chop the file first 
from pydub import AudioSegment

audio = AudioSegment.from_ogg(audio_file)

# PyDub handles time in milliseconds
ten_minutes = 10 * 60 * 1000

first_10_minutes = audio[:ten_minutes]
output_name = f"{ogg_file}_10.ogg"
first_10_minutes.export(output_name, format="ogg")


<_io.BufferedRandom name='processed_earning_call_morepen.ogg_10.ogg'>

<_io.BufferedReader name='processed_earning_call_morepen.ogg_10.ogg'>


In [ ]:
import json

In [ ]:
with open('morepen_earning_transcriptoni.json', 'w') as f:
    json.dump(transcription.__dict__, f)

In [ ]:
transcription   

Transcription(text="Ladies and gentlemen, good day and welcome to Morphine Laboratories Limited Q4 FY24 conference call, hosted by Motilal Oswal Financial Services Limited. As a reminder, all participant lines will be in the listen-only mode, and there will be an opportunity for you to ask questions after the presentation concludes. Should you need assistance during the conference call, please signal an operator by pressing star, then zero on your touch-tone phone. Please note that this conference is being recorded. I now hand the conference over to Mr. Tushar Manodani from Motilal Oswal Financial Services Limited. Thank you and over to you, Mr. Manodani. Thanks, Renju. Good afternoon and I welcome you all for the fourth quarter FY24 earnings call of Morphine Labs, hosted by Motilal Oswal Financial Services. From the management side, we have Mr. Sushil Suri, Chairman and Managing Director, Mr. Ajay Kumar Sharma, Chief Financial Officer, Mr. Nishant Doshi, Vice President, Corporate Fina

## Part two - processing the transcription - huge text

In [ ]:
def sentiment_analysis(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:

def action_item_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:

def key_points_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
def abstract_summary_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [ ]:
def meeting_minutes(transcription):
    abstract_summary = abstract_summary_extraction(transcription)
    key_points = key_points_extraction(transcription)
    action_items = action_item_extraction(transcription)
    sentiment = sentiment_analysis(transcription)
    return {
        'abstract_summary': abstract_summary,
        'key_points': key_points,
        'action_items': action_items,
        'sentiment': sentiment
    }

In [ ]:
# audio_file_path = "Earningscall.wav"

minutes = meeting_minutes(transcription.text)
print(minutes)

{'abstract_summary': "Morphine Laboratories Limited held its Q4 FY24 conference call, discussing the company's performance and future plans. The company reported a 20% annual revenue growth, from 1,424 crores to 1,704 crores, with a 16.5% increase in Q4. The medical devices segment, which contributes 26% of the company's revenue, grew by 35%, while the API business grew by 14%. The company's EBITDA doubled from 85 crores to 173 crores, and profit after tax increased by 150% from 38 crores to 97 crores. The company also reported a 100% increase in cash generation, from 83 crores to 168 crores. Morphine Labs is a debt-free company and is celebrating its 40th year in business. The company's main products are the BP monitor and glucose monitor, which are market leaders in their respective categories.", 'key_points': "1. The conference call was hosted by Motilal Oswal Financial Services Limited for Morphine Laboratories Limited's Q4 FY24 earnings.\n2. The management team present included Mr

## timestamps  for better representation

In [ ]:
from openai import OpenAI
client = get_openai_client()
client.timeout=50

def transcribe_audio_as_subtitles(audio_file_path):
    with open(audio_file_path, 'rb') as af:
        transcription = client.audio.transcriptions.create( model= "whisper-1", file=af,
                                                            response_format="srt",
                                                            )
    return transcription

def transcribe_audio_with_ts(audio_file_path):
    with open(audio_file_path, 'rb') as af:
        transcription = client.audio.transcriptions.create( model= "whisper-1", file=af,
                                                            response_format="verbose_json",
                                                            timestamp_granularities=["segment"]
                                                            )
    return transcription # not working as expected, fall back to creating srt or normal text.



In [ ]:
transcribed_json = transcribe_audio_as_subtitles(output_name)
transcribed_json

"1\n00:00:00,000 --> 00:00:06,880\nLadies and gentlemen, good day and welcome to Morphine Laboratories Limited Q4 FY24 conference\n\n2\n00:00:06,880 --> 00:00:13,120\ncall, hosted by Motilal Oswal Financial Services Limited. As a reminder, all participant lines\n\n3\n00:00:13,120 --> 00:00:18,080\nwill be in the listen-only mode, and there will be an opportunity for you to ask questions after\n\n4\n00:00:18,080 --> 00:00:22,320\nthe presentation concludes. Should you need assistance during the conference call,\n\n5\n00:00:22,320 --> 00:00:26,879\nplease signal an operator by pressing star, then zero on your touch-tone phone.\n\n6\n00:00:26,879 --> 00:00:31,840\nPlease note that this conference is being recorded. I now hand the conference over to Mr.\n\n7\n00:00:31,840 --> 00:00:37,200\nTushar Manodani from Motilal Oswal Financial Services Limited. Thank you and over to you,\n\n8\n00:00:37,200 --> 00:00:45,919\nMr. Manodani. Thanks, Renju. Good afternoon and I welcome you all for the fo

In [ ]:
with open('morepen_earning_transcripton_10.srt', 'w') as f:
    f.write(transcribed_json)-

In [ ]:
transcribed_json = transcribe_audio_with_ts(output_name)
transcribed_json

Transcription(text="Ladies and gentlemen, good day and welcome to Morphine Laboratories Limited Q4 FY24 conference call, hosted by Motilal Oswal Financial Services Limited. As a reminder, all participant lines will be in the listen-only mode, and there will be an opportunity for you to ask questions after the presentation concludes. Should you need assistance during the conference call, please signal an operator by pressing star, then zero on your touch-tone phone. Please note that this conference is being recorded. I now hand the conference over to Mr. Tushar Manodani from Motilal Oswal Financial Services Limited. Thank you and over to you, Mr. Manodani. Thanks, Renju. Good afternoon and I welcome you all for the fourth quarter FY24 earnings call of Morphine Labs, hosted by Motilal Oswal Financial Services. From the management side, we have Mr. Sushil Suri, Chairman and Managing Director, Mr. Ajay Kumar Sharma, Chief Financial Officer, Mr. Nishant Doshi, Vice President, Corporate Fina

In [ ]:
# done, takeaways -
## the above transcriptions cost us around $3-4 which is a lottt. Any cleaning before sending could save a lot of money
## deepmind seems to be better at this with streaming transcription available - get api key
## the ffmpeg utility is superb at compressing the audio 
## first compress then convert into chunks of 5-10 minutes, pass in a loop and append the transcriptionsi
## try prompts - maybe we can do speaker diariization there only

In [13]:
from datetime import datetime
from enums import tsFormats


def transcribe_audio_in_format(client,
                               audio_file_path,
                                format:tsFormats=tsFormats.JSON):
    with open(audio_file_path, 'rb') as af:
        print("calling open api whisper started...")
        st = datetime.utcnow()

        transcription = client.audio.transcriptions.create( model= "whisper-1", file=af,
                                                            response_format=format.value,
                                                            )
        print("calling open api end.")
        et = datetime.utcnow()
        total_time = et - st
        print("total time ", total_time.seconds, 'seconds')
        print("total tokens ", round(0.006*total_time.seconds))
    return transcription


In [15]:
from  dirs import *

f_chopped = 'earning_call_morepen_1.ogg'
transcription = transcribe_audio_in_format(client=get_openai_client(),
                                           audio_file_path=Path(CHOP_DIR,f_chopped))
transcription


calling open api whisper started...
calling open api end.
total time  27 seconds
total tokens  0


Transcription(text="Ladies and gentlemen, good day and welcome to Morphine Laboratories Limited Q4 FY24 conference call, hosted by Motilal Oswal Financial Services Limited. As a reminder, all participant lines will be in the listen-only mode, and there will be an opportunity for you to ask questions after the presentation concludes. Should you need assistance during the conference call, please signal an operator by pressing star, then zero on your touch-tone phone. Please note that this conference is being recorded. I now hand the conference over to Mr. Tushar Manodani from Motilal Oswal Financial Services Limited. Thank you and over to you, Mr. Manodani. Thanks, Renju. Good afternoon and I welcome you all for the fourth quarter FY24 earnings call of Morphine Labs, hosted by Motilal Oswal Financial Services. From the management side, we have Mr. Sushil Suri, Chairman and Managing Director, Mr. Ajay Kumar Sharma, Chief Financial Officer, Mr. Nishant Doshi, Vice President, Corporate Fina

In [16]:

type(transcription)

openai.types.audio.transcription.Transcription

In [17]:
transcription

Transcription(text="Ladies and gentlemen, good day and welcome to Morphine Laboratories Limited Q4 FY24 conference call, hosted by Motilal Oswal Financial Services Limited. As a reminder, all participant lines will be in the listen-only mode, and there will be an opportunity for you to ask questions after the presentation concludes. Should you need assistance during the conference call, please signal an operator by pressing star, then zero on your touch-tone phone. Please note that this conference is being recorded. I now hand the conference over to Mr. Tushar Manodani from Motilal Oswal Financial Services Limited. Thank you and over to you, Mr. Manodani. Thanks, Renju. Good afternoon and I welcome you all for the fourth quarter FY24 earnings call of Morphine Labs, hosted by Motilal Oswal Financial Services. From the management side, we have Mr. Sushil Suri, Chairman and Managing Director, Mr. Ajay Kumar Sharma, Chief Financial Officer, Mr. Nishant Doshi, Vice President, Corporate Fina

In [24]:
s = 'sdasd'
x = s ---
x

'_'

In [22]:
import json 
with open('f.json', 'w') as f:
    json.dump(transcription.__dict__,f)
print("Success")

Success
